## Establish connection with redshift

In [ ]:
#!/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip
#!pip install psycopg2-binary
#! pip install dask
import sys
import pandas as pd
#import dask.dataframe as dd

def redshift_connection(dbname, host, port, user, password):
    """
    This function connects with the redshift database.
    """
    import psycopg2 ## importing psycopg2
    ## creating connection using psycopg2
    conn = psycopg2.connect(dbname = dbname, host = host, port= port, user = user, password= password)
    return conn ## return connection

def data_execute(query, conn):
    """
    This function fetch the data using provided query from the database.
    """
    df = pd.read_sql_query(query, conn) ## reading data from the database
    return df 

## creating redshift connection with given data
conn = redshift_connection(dbname= 'prod_database', 
                           host='private-redshift-cluster-1.cw41yiij7sku.ap-south-1.redshift.amazonaws.com', 
                           port= '5338', 
                           user= 'kishlaya', 
                           password= 'Nove@2021')x

query = "select * from schema.table"
df = data_execute(query, conn)

## For dumping csv to s3


In [ ]:
 ### getting the access 
import boto3
import io
## creating connection with s3
s3_client = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY,aws_session_token=AWS_SESSION_TOKEN)

## dumping data
with io.StringIO() as csv_buffer:
    df02.to_csv(csv_buffer, index=False)
    response = s3_client.put_object(Bucket="subk-prod-raw-incremental-buckett", Key="Credit/df02.csv", Body=csv_buffer.getvalue())
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

## For dumping model to s3

In [ ]:
import tempfile
import boto3
import joblib

bucket_name = "subk-prod-raw-incremental-buckett"
key = "Credit/model1a.sav"
s3_client = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY,aws_session_token=AWS_SESSION_TOKEN)
# WRITE
with tempfile.TemporaryFile() as fp:
    joblib.dump(gsearch, fp)
    fp.seek(0)
    s3_client.put_object(Body=fp.read(), Bucket=bucket_name, Key=key)

## For getting data from s3

In [ ]:
 ### getting the access 
import boto3
import io
## creating connection with s3
s3_client = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY,aws_session_token=AWS_SESSION_TOKEN)

## getting data
response = s3_client.get_object(Bucket="subk-prod-raw-incremental-buckett", Key="Credit/RBL_FSM/los_history_02.csv")
import pandas as pd
data = pd.read_csv(response['Body'])

## Reading multiple excel from s3 and stacking rows in a single df

In [ ]:
s3 = boto3.resource('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY,aws_session_token=AWS_SESSION_TOKEN)
bucket = s3.Bucket('bucket_name')
prefix_objs = bucket.objects.filter(Prefix="path_to_req_folder")
prefix_df = pd.DataFrame()
for obj in prefix_objs:
    try:
        key = obj.key
        body = obj.get()['Body'].read()
        temp = pd.read_excel(body,engine='openpyxl')#use engine='pyxlsb' for xlsb files
        prefix_df=prefix_df.append(temp)
        print('done for month',str(obj.key))
    except Exception as e:
        print(e,key)

## Reading from S3 using aws wrangler

In [8]:
import awswrangler as wr
path="s3://bucket/path/filename"
session = boto3.session.Session(aws_access_key_id=AWS_ACCESS_KEY_ID,\
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY,\
                         aws_session_token=AWS_SESSION_TOKEN)
df= wr.s3.read_excel(path=path,boto3_session=session,engine='openpyxl')

## Getting all files names

In [94]:
bucket = "subk-prod-raw-incremental-buckett"
folder = "Credit/hostorical_data/rbl/F.Y. 2016-2017/FY16_17_Demand"
s3 = boto3.resource("s3", aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                         aws_session_token=AWS_SESSION_TOKEN)
s3_bucket = s3.Bucket(bucket)
files_in_s3 = [f.key.split(folder + "/")[1] for f in s3_bucket.objects.filter(Prefix=folder).all()]

## Listing all schemas and tables of redshift

In [ ]:
#select * from pg_namespace; - see all schemas
#set search_path to '$user', 'public', 'req_schema_name'; - run this query in redshift to choose ur schema.
#SELECT DISTINCT tablename FROM pg_table_def ## this will give all schema names+tables names...

##see tables from seclected schema.
query = " SELECT DISTINCT tablename FROM pg_table_def WHERE schemaname = 'dw_credit'; "
c = data_execute(query, conn)

In [ ]:
#Getting top 10 rows from multiple tables in a dict.
d={}
for i in c.tablename:
    query = " SELECT top 10 * FROM dw_credit.{0}".format(i)
    d[i]=data_execute(query, conn)
    print('table {0} is done'.format(i))    

In [35]:
#Putting multiple dfs to different sheets of a excel file.
with pd.ExcelWriter("all_tables.xlsx") as writer:
    for i in d.keys():
        d[i].to_excel(writer, sheet_name='{0}'.format(i[0:30]), index=False)       